In [2]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel, QFileDialog, QMessageBox
from PyQt5.QtGui import QPixmap, QImage, QFont
from PyQt5.QtCore import QTimer
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [3]:
class SignLanguageApp(QMainWindow):
    def __init__(self):
        super(SignLanguageApp, self).__init__()
        self.setWindowTitle("Sign Language Detection")
        self.setGeometry(100, 100, 800, 600)

        #### Styling background 
        self.setStyleSheet("""
            QMainWindow {
                background: qlineargradient(
                    x1: 0, y1: 0, x2: 1, y2: 1,
                    stop: 0 #f7b733, stop: 1 #fc4a1a
                );
            }
        """)

        self.title_label = QLabel("Sign Language Detection", self)
        self.title_label.setGeometry(100, 10, 600, 60)
        self.title_label.setStyleSheet("""
            QLabel {
                color: #ffffff;
                font-size: 28px;
                font-weight: bold;
                text-align: center;
                text-shadow: 2px 2px 4px #000000;
            }
        """)
        self.title_label.setFont(QFont("Arial", 28))

        #### Creating and styling upload button  
        self.upload_button = QPushButton("Upload Image", self)
        self.upload_button.setGeometry(50, 80, 220, 50)
        self.upload_button.setStyleSheet("""
            QPushButton {
                background-color: #FF6347;
                color: white;
                font-size: 16px;
                border-radius: 10px;
            }
            QPushButton:hover {
                background-color: #FF4500;
            }
        """)
        self.upload_button.clicked.connect(self.upload_image)

        #### Creating real-time detection button 
        self.video_button = QPushButton("Real-time Detection", self)
        self.video_button.setGeometry(300, 80, 220, 50)
        self.video_button.setStyleSheet("""
            QPushButton {
                background-color: #4682B4;
                color: white;
                font-size: 16px;
                border-radius: 10px;
            }
            QPushButton:hover {
                background-color: #4169E1;
            }
        """)
        self.video_button.clicked.connect(self.start_real_time_detection)

        self.image_label = QLabel(self)
        self.image_label.setGeometry(50, 150, 640, 400)
        self.image_label.setStyleSheet("""
            QLabel {
                background-color: #ffffff;
                border: 2px solid #d3d3d3;
                border-radius: 15px;
            }
        """)
        self.image_label.setScaledContents(True)

        #### styling prediction label
        self.prediction_label = QLabel("Prediction: ", self)
        self.prediction_label.setGeometry(50, 560, 700, 40)
        self.prediction_label.setStyleSheet("color: #ffffff; font-size: 20px; font-weight: bold;")
        self.prediction_label.setFont(QFont("Arial", 20))

        #### here we will load the traines model
        self.model = load_model('Sign_Language_CNN_Model.keras')
        self.class_names = ['1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 
                            'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 
                            'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'best of luck', 'i love you', 'space']

        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)
        self.cap = None

    def upload_image(self):
        """Opens a file dialog to upload an image and display it on the GUI."""
        options = QFileDialog.Options()
        file_path, _ = QFileDialog.getOpenFileName(self, "Select Image", "", "Image Files (*.png *.jpg *.jpeg)", options=options)
        
        if file_path:
            pixmap = QPixmap(file_path)
            scaled_pixmap = pixmap.scaled(self.image_label.width(), self.image_label.height(), aspectRatioMode=1)
            self.image_label.setPixmap(scaled_pixmap)

            #### For model  prediction we will preprocess the image
            img = image.load_img(file_path, target_size=(64, 64))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = img_array / 255.0

            prediction = self.model.predict(img_array)
            predicted_class = np.argmax(prediction, axis=1)[0]

            predicted_label = self.class_names[predicted_class % len(self.class_names)]
            self.prediction_label.setText(f"Prediction: {predicted_label}")
        else:
            QMessageBox.warning(self, "Warning", "No image selected.")

    def start_real_time_detection(self):
        """Start the camera for real-time sign language detection."""
        self.cap = cv2.VideoCapture(0)  
        if not self.cap.isOpened():
            QMessageBox.warning(self, "Warning", "Unable to access the camera.")
            return
        
        self.timer.start(30) 

    def update_frame(self):
        """Capture frames from the camera and predict sign language."""
        ret, frame = self.cap.read()
        if ret:
            #### for displaying convert the frame to RGB 
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            #### For the model prediction we will preprocess the frame 
            img_array = cv2.resize(rgb_frame, (64, 64))  # Resize to (64, 64)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = img_array / 255.0

            #### PREDICTION
            prediction = self.model.predict(img_array)
            predicted_class = np.argmax(prediction, axis=1)[0]

            predicted_label = self.class_names[predicted_class % len(self.class_names)]
            self.prediction_label.setText(f"Prediction: {predicted_label}")

            #### Displaying the video feed
            h, w, ch = rgb_frame.shape
            bytes_per_line = ch * w
            qt_image = QImage(rgb_frame.data, w, h, bytes_per_line, QImage.Format_RGB888)
            self.image_label.setPixmap(QPixmap.fromImage(qt_image))
        else:
            self.timer.stop()
            self.cap.release()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = SignLanguageApp()
    window.show()
    sys.exit(app.exec_())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


SystemExit: 0

C:\Users\jhara\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
